In [9]:
import pandas as pd
import numpy as np

In [10]:
import os
dirname = os.getcwd()
print(dirname)

/home/jupyterlab/pipeline-202102


In [11]:
os.environ['MPLCONFIGDIR'] = dirname
import matplotlib

In [12]:
import xlsxwriter
import xlrd

In [13]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-02-08T14:11:23.833062


In [14]:
print(my_date.strftime('%Y-%m-%dT%H:%M:%S.%f%z'))

2021-02-08T14:11:23.833062


In [15]:
runid = "R" + my_date.strftime('%Y%m%d%H%M%S%f%z')
runid

'R20210208141123833062'

In [16]:
filename = dirname + '/Data.xlsx'
filedf = pd.read_excel(filename,engine='openpyxl', sheet_name=None, header=None)

In [17]:
sheetnames = list(filedf.keys())

In [18]:
def validate_date(date_text):
    try:
        format_string = '%Y-%m-%d %H:%M:%S'
        datetime.strptime(date_text, format_string)
        return "OK"
    except ValueError:
        return None

def GetDataType(row_in):
    valueasstring = str(row_in['CellValue'])
    if validate_date(valueasstring) != None:
        return 'Date'
    elif valueasstring.isnumeric():
        return 'Number'
    else:
        return 'Text'

def add1_2row(row_in):
    return row_in['RowID'] + 1


def add1_2column(row_in):
    return row_in['ColumnID'] + 1

In [19]:
celldbstagedf = pd.DataFrame()
sheetdf = filedf['DataMap'].copy()
sheetdf["RowID"] = sheetdf.index
#sheetdf

In [20]:
for currentsheetname in sheetnames:

    sheetdf = filedf[currentsheetname].copy()
    sheetdf["RowID"] = sheetdf.index
    celldb = pd.melt(sheetdf, id_vars=['RowID'])
    celldb = celldb.rename(columns={"variable": "ColumnID", "value": "CellValue"})
    celldb["CellType"] = "Unset"
    celldb["FileName"] = runid
    celldb["SheetName"] = currentsheetname
    
    celldb["ColumnID"] = celldb["ColumnID"]

    try:
        celldb["CellType"] = celldb.apply(GetDataType, axis=1)
        celldb["RowID"] = celldb.apply(add1_2row, axis=1)
        celldb["ColumnID"] = celldb.apply(add1_2column, axis=1)

    except:

        celldb["CellType"] = 'unknown'
        celldb["RowID"] = 0
        celldb["ColumnID"] = 0

    celldbstagedf = celldbstagedf.append(celldb, ignore_index=True)

    celldb = pd.DataFrame()



In [ ]:
fullcelldb = celldbstagedf.copy()

In [ ]:
Cells = pd.DataFrame()
Cells = fullcelldb[(fullcelldb['SheetName'] == 'UniclassComplexCode')].copy()
Cells.head()

In [ ]:
##Below extracts a Sheet from the cell database and reshapes it to it original tabular form.
Table = pd.DataFrame()
Table = Cells.pivot_table(index = ['RowID'], values = 'CellValue',columns = 'ColumnID', aggfunc = 'max')
Headings = Table.iloc[0]
HeadingsList = list(Headings)
print(HeadingsList)

In [ ]:
HeadingsListTuple = enumerate(HeadingsList,start=1)
HeadingsListDict = dict(HeadingsListTuple)
print(HeadingsListDict)
Table = Table.rename(columns=HeadingsListDict)
Table = Table.drop([1])
Table.head()

SELECT
@DataSubmissionRunID = [SubmissionRunID]
,@DataDrive = [Folder1]
,@DataEnvironment = [Folder2]
,@DataConveyorStage = [Folder3]
,@DataSubject= [Folder4]
,@DataContract = [Folder5]
,@DataDate = [Folder6]
,@DataPMU = [Folder7]
,@DataFolder8 = [Folder8]
,@DataFolder9 = [Folder9]
,@DataFolder10 = [Folder10]
,@dag  = [Folder7]
FROM [Process].[Files] where [SubmissionRunID] = @SubRunID; -- and Folder7 = @dag

In [ ]:
select SheetName,CellType,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID,
cast(CellValue as varchar(max)) as CellValue into Working.DataCellData 
from [Data].[Cells]
where [SubmissionRunID] = @SubRunID;

In [ ]:
select [SubmissionRunID],[Folder8],[FileName] into Working.SpecList
from
(select
[SubmissionRunID],
[Folder6],
[Folder8],
[FileName],
max(Folder8) over (Partition by [FileName]) LatestVersion
from [Process].[Files]
where Folder4 = '100_ReferenceData' and Folder6 = @DataPMU) g
where [Folder8] = LatestVersion;

In [ ]:
select  @DataSpecRunID = [SubmissionRunID] from Working.SpecList where [FileName] = 'DataSpec.xlsx';
select  @DataMapRunID = [SubmissionRunID] from Working.SpecList where [FileName] = 'DataMap.xlsx';

select
SheetName,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID,
cast(CellValue as varchar(max)) as CellValue
into Working.DataSpecCells 
from [Data].[Cells]
where [SubmissionRunID] = @DataSpecRunID;


In [ ]:
select * into Working.DataSetCells from Working.DataSpecCells where SheetName = 'DataSet' and RowID > 1;
select * into Working.DataSetColumns from Working.DataSpecCells where SheetName = 'DataSet' and RowID = 1;
select cel.*, col.CellValue as FieldName into Working.DataSetDataSet from Working.DataSetCells cel, Working.DataSetColumns col where cel.ColumnID = col.ColumnID;

In [ ]:


DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.DataSetDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;

In [ ]:
DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

select * into Working.celldata from Working.DataSetDataSet;

In [ ]:


------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

In [ ]:

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.DataSetTable; 

select * into Working.DataSetTable from Working.pivoteddata where [DataSetID] = @Dataset_in;

DROP TABLE IF EXISTS Working.pivoteddata;

-----------------------

In [ ]:
select distinct @ReferenceSheetName = SheetName, @ReferenceTextStart = ReferenceTextStart from working.DataSetTable;

select @AnchorColumn = ColumnID, @AnchorRow = RowID from working.DataCellData
where
SheetName = @ReferenceSheetName and CellValue = @ReferenceTextStart;


In [ ]:
------  data map ---------

select SheetName,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID, cast(CellValue as varchar(max)) as CellValue
into Working.DataMapCells 
from [Data].[Cells]
where [SubmissionRunID] = @DataMapRunID and SheetName = 'DataMap' and RowID > 1;

select SheetName,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID, cast(CellValue as varchar(max)) as CellValue
into Working.DataMapColumns 
from [Data].[Cells]
where [SubmissionRunID] = @DataMapRunID and SheetName = 'DataMap' and RowID = 1;

select cel.*, col.CellValue as FieldName into Working.DataMapDataSet from Working.DataMapCells cel, Working.DataMapColumns col where cel.ColumnID = col.ColumnID;


In [ ]:

---------------------

DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.DataMapDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;


In [ ]:
DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

select * into Working.celldata from Working.DataMapDataSet;

In [ ]:

------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.DataMapTable; 

select * into Working.DataMapTable from Working.pivoteddata  where [DataSetID] = @Dataset_in;

DROP TABLE IF EXISTS Working.pivoteddata;

--select FieldID, ColumnName, ColumnType from Working.DataMapTable

commit transaction;

In [ ]:
------  data meta data ---------
begin transaction;

DROP TABLE if exists Working.MetaDataColumns;
DROP TABLE if exists Working.MetaDataCells;
DROP TABLE if exists Working.MetaDataDataSet;
DROP TABLE if exists Working.MetaDataTable;

select SheetName,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID, cast(CellValue as varchar(max)) as CellValue
into Working.MetaDataCells 
from [Data].[Cells]
where [SubmissionRunID] = @DataMapRunID and SheetName = 'MetaData' and RowID > 1;

select SheetName,try_cast(ColumnID as Integer) as ColumnID,Try_cast([RowID] as Integer) as RowID, cast(CellValue as varchar(max)) as CellValue
into Working.MetaDataColumns 
from [Data].[Cells]
where [SubmissionRunID] = @DataMapRunID and SheetName = 'MetaData' and RowID = 1;

select cel.*, col.CellValue as FieldName into Working.MetaDataDataSet from Working.MetaDataCells cel, Working.MetaDataColumns col where cel.ColumnID = col.ColumnID;



In [ ]:
--------------------

DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.MetaDataDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;

DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

--select 'DataSetDataSet';

--select * from Working.DataSetDataSet;


select * into Working.celldata from Working.MetaDataDataSet;

In [ ]:
------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.MetaDataTable; 

select * into Working.MetaDataTable from Working.pivoteddata  where [DataSetID] = @Dataset_in;

DROP TABLE IF EXISTS Working.pivoteddata;



SELECT 
      @TableDatabase = [Database]
      ,@TableDatasetID = [DataSetID]
      ,@TableSchema = [Schema]
      ,@TableName = [TableName]
  FROM [Working].[MetaDataTable]
  where [DataSetID] = @Dataset_in

commit transaction;

-----------------------


------------------------

begin transaction;

DROP TABLE IF EXISTS Working.PublishDataSet; 

select
--ds.[RowID]
ds.[ColumnFromReferenceText]
,ds.[DataSetID]
,ds.[FieldDescription]
,ds.[FieldID] as FieldName
,ds.[FieldPosition]
,ds.[ReferenceTextEnd]
,ds.[ReferenceTextStart]
,ds.[RowFromReferenceText]
,ds.[SheetName]
,ds.[TableOrientation]
--,dc.[SheetName]
,dc.[CellType]
,dc.[ColumnID]
,dc.[RowID]
,dc.[CellValue]
into Working.PublishDataSet from
working.DataSetTable ds, working.DataCellData dc
where
dc.SheetName = ds.SheetName and
dc.ColumnID = ds.ColumnFromReferenceText + @AnchorColumn and dc.RowID > ds.RowFromReferenceText + @AnchorRow;


--------------------- prepare for pivot -------------------------------------------------
DROP TABLE IF EXISTS Working.fieldnames; 

select FieldName into Working.fieldnames from Working.PublishDataSet;

set  @fieldnameslist = NULL

SELECT @fieldnameslist = ISNULL(@fieldnameslist + ',','') + QUOTENAME(FieldName)
FROM (SELECT DISTINCT FieldName FROM Working.fieldnames) AS FieldName;

DROP TABLE if exists Working.celldata; 
DROP TABLE if exists Working.pivoteddata;

--select 'DataSetDataSet';

--select * from Working.DataSetDataSet;


select * into Working.celldata from Working.PublishDataSet;

------- Do Pivot ----------

--Prepare the PIVOT query using the dynamic 
SET @DynamicPivotQuery = 
N'select * into Working.pivoteddata from (select RowID,FieldName,CellValue from Working.celldata) src
PIVOT(max(CellValue) for FieldName in ('+@fieldnameslist+')) piv;';
--Execute the Dynamic Pivot Query

--select @DynamicPivotQuery;

EXEC sp_executesql @DynamicPivotQuery;

---- Populate Pivoted Table

DROP TABLE IF EXISTS Working.PublishTable; 


In [ ]:


--------
set @createtable = NULL;

SELECT @createtable = ISNULL(@createtable + ',','') + CreateField
FROM (SELECT '['+ColumnName+']' + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]) AS CreateField;

SELECT @columnlist = ISNULL(@columnlist + ',','') + FieldName
FROM (SELECT '['+ColumnName+']' FieldName FROM [Working].[DataMapTable]) AS FieldName;

--SELECT ColumnName + ' ' + ColumnType as CreateField FROM [Working].[DataMapTable]

-- @TableSchema = [Schema]
-- @TableName = [TableName]

set @createtable = 'create table ' + @TableSchema +'.' + @TableName + '(' +@createtable + ')'

--select @createtable;

set @SQLCmnd = 'DROP TABLE IF EXISTS ' + @TableSchema +'.' + @TableName

EXEC sp_executesql @SQLCmnd;

EXEC sp_executesql @createtable;

set @SQLCmnd = 'insert into ' + @TableSchema +'.' + @TableName + ' select ' + @columnlist + ' from Working.pivoteddata;'

--select @SQLCmnd

EXEC sp_executesql @SQLCmnd;

DROP TABLE IF EXISTS Working.pivoteddata;

commit transaction;

--select * from Working.PublishTable;

GO